In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd


In [2]:
## Load the trained model, scaler and onehot
model=load_model('model.h5')

## Load encoder and scalae 
with open('pickel_file/lable_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
with open('pickel_file/onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)
with open('pickel_file/scalar.pkl','rb') as file:
    scalar=pickle.load(file)

In [4]:
## Example data 
input_data={
    'CreditScore':600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000 
}
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [7]:
geo_encoded=onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\Data Science\Bank-Churn-Modeling-using-ANN\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [9]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [10]:
# Encode my categorical vairlables 
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [13]:
## Concatination one hot encoded data 
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)


KeyError: "['Geography'] not found in axis"

In [14]:
## Scaling the input data 
input_scaled=scalar.transform(input_df)

In [15]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
## Predict Churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 118ms/step


array([[0.03751456]], dtype=float32)

In [18]:
prediction_proba=prediction[0][0]
prediction_proba

0.037514564

In [19]:
if prediction_proba>0.5:
    print("The Customer is likely to Churn")
else:
    print("The Customer is not likely to Churn")

The Customer is not likely to Churn
